#  Sales Trend Analysis

This analysis focuses on understanding how sales have evolved over time to identify patterns, trends, and seasonality in customer purchases. These insights support better business decisions in marketing, inventory, and forecasting.

## What We Will Explore:

# 1. Time-Based Sales Aggregation

Monthly Sales: Total revenue per month.

Daily Sales: Total revenue per day.

# 2. Smoothing & Trend Detection

3-Month Moving Average: Helps smooth short-term fluctuations and highlight overall sales trends.

# 3. Visualizations

To better understand sales trends over time, we will use:

Matplotlib: Line chart of monthly sales to show overall trends.

Matplotlib: Line chart with a 3-month moving average overlay to visualize smoothed sales trends.

Plotly: Interactive line chart of daily sales to detect short-term fluctuations and anomalies.